In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-tanyas71lsu')

In [14]:
dem = ee.Image('USGS/3DEP/10m')

geom = ee.Geometry.Point([-91.0989573,30.3529013])
geom_col = ee.FeatureCollection([geom])
elev = dem.sampleRegions(geom_col).getInfo()
#print(elev)
#elev
elev['features'][0]['properties']['elevation']

3.220296621322632

In [20]:
import pandas as pd
table = pd.read_csv(r"C:\Users\thany\Documents\programming\geog4057_TanyaSutthanonkul\TermProject\boundary.csv")
table.head()

,col,row,X,Y
0,4871,174,699102.887792,186780.445813
1,4871,174,699102.887792,186780.445813
2,4872,174,699105.887419,186780.445813
3,4870,175,699099.888166,186777.446186
4,4873,174,699108.887046,186780.445813


In [22]:
import arcpy
ra1 = arcpy.Raster(r"C:\Users\thany\Documents\programming\geog4057_TanyaSutthanonkul\TermProject\flood_2class.tif") 
print(ra1.spatialReference.factoryCode)

32119


In [24]:
import geopandas 
gdf = geopandas.GeoDataFrame(table)
gdf.set_geometry( geopandas.points_from_xy(gdf['X'], gdf['Y']), inplace=True, crs=f'EPSG:{ra1.spatialReference.factoryCode}')

In [25]:
gdf.to_file(r'C:\Users\thany\Documents\programming\geog4057_TanyaSutthanonkul\TermProject\boundary.shp')

In [26]:
shapefile = r'C:\Users\thany\Documents\programming\geog4057_TanyaSutthanonkul\TermProject\boundary.shp'

arcpy.management.AddField(shapefile,'elevation',field_type='FLOAT')

<Result 'C:\\Users\\thany\\Documents\\programming\\geog4057_TanyaSutthanonkul\\TermProject\\boundary.shp'>

In [27]:
geom_list = []
with arcpy.da.SearchCursor(shapefile,['SHAPE@XY'],spatial_reference=4326) as cursor:
    for row in cursor:
        X,Y = row[0]
        geom = ee.Geometry.Point([X,Y])
        geom_list.append(geom)
geom_col = ee.FeatureCollection(geom_list)
elev = dem.sampleRegions(geom_col).getInfo().get('features')

In [29]:
i = 0
with arcpy.da.UpdateCursor(shapefile,['elevation']) as cursor:
    for row in cursor:
        elevation = elev[i]['properties']['elevation']
        row[0] = elevation
        cursor.updateRow(row)
        i += 1

####### test by put shp to arcgis pro and see that we have elevation field values